In [71]:
!pip install nbformat>=4.2.0

In [67]:
!pip install plotly

In [75]:
!pip install ipykernel

In [1]:
import pandas as pd 
import numpy as np
import scipy
import io

In [2]:
data_path = r"C:\Users\anapa\Documents\MATLAB\ppgeb_masters\data\tempo_piscada\teste_piscada_02_12_4segundos.txt"

In [3]:
def read_dataset(data_path):
    data_list = []
    with io.open(data_path, mode="r", encoding="utf-8") as f:
        for line in f:
            data_list.append(line.rstrip().split())
    return data_list

def clean_dataset(data_list):
    df = pd.DataFrame(data_list)
    df.columns=df.iloc[0] 
    df = df.iloc[1:]
    df.iloc[-1,-1] = df.iloc[-1,3]
    df.USER = df.USER.replace("START_RECORDING", "START_RECORDING 100").replace("STOP_EYETRACKER", "STOP_EYETRACKER 200")
    df = df.loc[df["USER"] != "CLIENT2_READY"]
    df = df.loc[df.DATATYPE == "SMP"]
    df["BKID"] = np.where(df["BKID"] != "0", 500, 0)
    df = df.loc[df["USER"].str.isnumeric()]
    df["USER"] = df["USER"].astype(int)
    return df

In [4]:
data_list = read_dataset(data_path)
df = clean_dataset(data_list)

In [5]:
pd.set_option("display.max_columns", None)

In [6]:
df = df[["TIME", "BKID", "USER"]]

In [9]:
df

,TIME,BKID,USER
139,1519.53821,0,55
140,1519.57556,0,55
142,1519.59448,0,54
143,1519.60547,0,54
144,1519.62732,0,54
...,...,...,...
23895,1796.74023,0,37
23899,1796.77417,0,69
23901,1796.79041,0,37
23902,1796.80518,0,37


In [7]:
import plotly.express as px
import numpy as np

In [20]:
cut = 800

fig = px.line(df.iloc[:-cut], x='TIME', y="USER")

fig.add_scatter(x=df['TIME'][:-cut], y=df['BKID'][:-cut], mode='lines')
fig.show()

In [21]:
mean = df["USER"][:-800].mean()
std = np.std(df["USER"].values)
print('mean of the dataset is', mean)
print('std. deviation is', std)

mean of the dataset is 79.13910568359195
std. deviation is 108.9906382833864


In [31]:
threshold = 1.5
outlier = []
for t, i in zip(df["TIME"].values, df["USER"].values):
    z = (i-mean)/std
    if z > threshold:
        outlier.append(t)

In [32]:
def is_outlier(val):
    if val in outlier:
        return 500
    else:
        return 0

In [33]:
df["OUTLIER"] = df["TIME"].apply(is_outlier)

In [46]:
cut = 800

fig = px.line(df.iloc[:-cut], x='TIME', y="USER")

fig.add_scatter(x=df['TIME'][:-cut], y=df['BKID'][:-cut], mode='lines')
fig.add_scatter(x=df['TIME'][:-cut], y=df['OUTLIER'][:-cut]* -1, mode='lines')
fig.show()

In [120]:
df.shape[0]

11691

In [154]:
 420 / 11691

0.0359250705671029

In [138]:
df.loc[(df["BKID"] == 500) & (df["OUTLIER"] == 500)].shape[0]

59

In [132]:
df.loc[df["BKID"] == 500].shape[0]

1389

Temos 11,691 pontos de dados.  
Coletamos aprox. 60 pontos de dados por segundo.  
Temos uma coleta de aprox. 195 segundos ou pouco mais de 3 min.  
A piscada foi frequente a cada 4 segundos, logo, houveram aprox. 48 piscadas durante o experimento. 
Piscadas identificadas pelo ET correspondem a 1,348 dos dos 11,691 dos dados; ou 11%  
Piscadas identificadas como outliers nos dados de EEG corresponderam a 420, ou 3% (1/3 da quantidade de piscadas identificadas pelo ET).   
Houve sobreposição da piscada identificada em EEG e ET em 59 pontos de dados, ou 0.5%

In [157]:
(59/11616) * 100

0.5079201101928374

In [136]:
df.loc[df["OUTLIER"] == 500].shape[0]

420

In [121]:
df.shape[0]/60

194.85

In [123]:
194 / 4

48.5

In [122]:
195/60

3.25

In [57]:
df  = df.reset_index().reset_index().rename(columns={"level_0":"dados"}).set_index("index")

In [83]:
df["dados"] = df["dados"].astype(int)

In [98]:
df["segundos"] = (df["dados"]/60).round()

In [112]:
df["segundo_com_piscada"] = np.where((df["segundos"] % 4 == 0), 1, 0)

In [146]:
df["piscada"] = np.where(((df["segundos"] % 4 <= 5) & (df["BKID"] != 0)), 1, 0)

In [150]:
df.loc[df["segundos"] == 15][:20]

,dados,TIME,BKID,USER,OUTLIER,segundos,piscada,segundo_com_piscada
index,,,,,,,,
1693,871,1537.79321,0,72,0,15.0,0,0
1695,872,1537.80896,0,40,0,15.0,0,0
1697,873,1537.82288,0,70,0,15.0,0,0
1699,874,1537.84314,0,20,0,15.0,0,0
1700,875,1537.85559,0,20,0,15.0,0,0
1702,876,1537.87781,0,49,0,15.0,0,0
1704,877,1537.89014,0,97,0,15.0,0,0
1706,878,1537.90491,0,193,0,15.0,0,0
1708,879,1537.92322,0,123,0,15.0,0,0


In [108]:
df.loc[df["piscada"] == 1].describe().round()

,dados,BKID,USER,OUTLIER,segundos,piscada
count,417.0,417.0,417.0,417.0,417.0,417.0
mean,5829.0,500.0,98.0,28.0,97.0,1.0
std,3121.0,0.0,126.0,114.0,52.0,0.0
min,490.0,500.0,0.0,0.0,8.0,1.0
25%,3129.0,500.0,35.0,0.0,52.0,1.0
50%,6010.0,500.0,68.0,0.0,100.0,1.0
75%,8660.0,500.0,108.0,0.0,144.0,1.0
max,11498.0,500.0,771.0,500.0,192.0,1.0


In [115]:
df.loc[(df["OUTLIER"] == 500)]

,dados,TIME,BKID,USER,OUTLIER,segundos,piscada,segundo_com_piscada
index,,,,,,,,
452,263,1524.10229,0,596,500,4.0,0,1
454,264,1524.11169,0,499,500,4.0,0,1
455,265,1524.13525,0,499,500,4.0,0,1
540,307,1524.94287,500,283,500,5.0,0,0
542,308,1524.95898,500,401,500,5.0,0,0
...,...,...,...,...,...,...,...,...
23114,11328,1789.53259,0,299,500,189.0,0,0
23116,11329,1789.54431,0,332,500,189.0,0,0
23439,11482,1792.75037,0,646,500,191.0,0,0


In [116]:
df.loc[(df["OUTLIER"] == 500) & (df["segundo_com_piscada"] == 1)]

,dados,TIME,BKID,USER,OUTLIER,segundos,piscada,segundo_com_piscada
index,,,,,,,,
452,263,1524.10229,0,596,500,4.0,0,1
454,264,1524.11169,0,499,500,4.0,0,1
455,265,1524.13525,0,499,500,4.0,0,1
1855,930,1539.40271,0,252,500,16.0,0,1
1857,931,1539.41980,0,250,500,16.0,0,1
...,...,...,...,...,...,...,...,...
22949,11257,1787.91846,0,434,500,188.0,0,1
22951,11258,1787.93445,0,538,500,188.0,0,1
22953,11259,1787.95129,0,444,500,188.0,0,1


In [118]:
df.loc[(df["BKID"] == 500) & (df["segundo_com_piscada"] == 1)]

,dados,TIME,BKID,USER,OUTLIER,segundos,piscada,segundo_com_piscada
index,,,,,,,,
923,490,1528.54700,500,72,0,8.0,1,1
925,491,1528.55847,500,73,0,8.0,1,1
926,492,1528.57349,500,73,0,8.0,1,1
927,493,1528.60498,500,73,0,8.0,1,1
928,494,1528.61438,500,73,0,8.0,1,1
...,...,...,...,...,...,...,...,...
23484,11494,1793.13611,500,178,0,192.0,1,1
23486,11495,1793.15137,500,140,0,192.0,1,1
23488,11496,1793.15833,500,170,0,192.0,1,1


In [117]:
df.loc[(df["OUTLIER"] == 500) & (df["segundo_com_piscada"] != 1)]

,dados,TIME,BKID,USER,OUTLIER,segundos,piscada,segundo_com_piscada
index,,,,,,,,
540,307,1524.94287,500,283,500,5.0,0,0
542,308,1524.95898,500,401,500,5.0,0,0
544,309,1524.97559,500,809,500,5.0,0,0
546,310,1524.99194,500,683,500,5.0,0,0
548,311,1525.00818,0,316,500,5.0,0,0
...,...,...,...,...,...,...,...,...
23112,11327,1789.51660,0,275,500,189.0,0,0
23114,11328,1789.53259,0,299,500,189.0,0,0
23116,11329,1789.54431,0,332,500,189.0,0,0


In [119]:
df.loc[(df["BKID"] == 500) & (df["segundo_com_piscada"] != 1)]

,dados,TIME,BKID,USER,OUTLIER,segundos,piscada,segundo_com_piscada
index,,,,,,,,
273,117,1521.67407,500,44,0,2.0,0,0
274,118,1521.68982,500,44,0,2.0,0,0
275,119,1521.70605,500,44,0,2.0,0,0
276,120,1521.72253,500,44,0,2.0,0,0
277,121,1521.73889,500,44,0,2.0,0,0
...,...,...,...,...,...,...,...,...
23281,11400,1790.94092,500,43,0,190.0,0,0
23283,11401,1790.95801,500,71,0,190.0,0,0
23285,11402,1790.97266,500,27,0,190.0,0,0


In [ ]:
# 420 vezes a piscada foi identificada nos dados de EEG. Quanto isso diz respeito 